# WORK IN PROGRESS

# Kvalita kódu
Většina poznámek, které nalezneme v tomto repozitáři, se točí kolem otázky jak něco udělat. Text, který právě čteme, má zaměření jiné - věnuje se kvalitě kódu a jeho udržitelnosti. Tj. jde nám o to, jak napsat kód tak, aby se v něm některý náš kolega (anebo třeba i my sami po půl roce) vyznal a aby byl kód co možná nejméně náročný na údržbu. 

## Virtuální prostředí
Člověku nic nebrání nainstalovat si nejprve Python a potom skrze pip i různorodé balíčky. Dřív nebo později ale dojde k situaci, kdy projekt A potřebuje balíček o verzi 0.20 a ne novější, zatímco projekt B stojí a padá s totožným balíčkem, ale ve verzi 1.25 a ne starší. Co s tím potom dělat? V závislosti na úloze, na které programátor zrovna pracuje, balíčky přeinstalovávat? Z toho by byl jen jeden velký zmatek a ještě by se přitom mohly nabořit verze dalších balíčků. Správně řešení spočívá ve využití virtuálních prostředí.  
Virtuální prostředí vytvoříme příkazem (puštěným v příkazovém řádku)
```
python -m venv environment
```
Zde říkáme "hlavní" instalaci pythonu, že chceme pustit balíček [venv](https://docs.python.org/3/library/venv.html) jako skript (proto parametr -m) s parametrem environment. Environment zde nereprezentuje klíčové slovo, ale jméno nového virtuálního prostředí a současně adresáře, který vznikne v pracovním adresáři příkazového řádku. Následně se do nově vytvořeného virtuálního prostředí přepneme příkazem
```
environment\Scripts\activate
```
resp. pokud jsme na Linuxu a nikoli na Windowsech
```
source environment\bin\activate
```
V tento okamžik už se příkazy **python** a **pip** nebudou vázat na python.exe a pip.exe bydlící v instalačním adresáři Pythonu, nýbrž na stejnojmené exe soubory ležící ve virtuálním prostředí. Když budeme nyní instalovat balíčky, uloží se (a budou viditelné v) jen do aktivního virtuálního prostředí a nebudou nijak interferovat s obsahem jiných virtuálních prostředí anebo s balíčky nainstalovanými v rámci základní pythoní instalace. Pokud chceme virtuální prostředí opustit, napíšeme do konzole příkaz
```
deactivate
```
Nikdy do adresáře environment (resp. jeho ekvivalentu, pokud jsme zvolili jiné jméno) explicitně neukládáme vlastní soubory. Onen adresář se totiž neverzuje (je na to moc velký a nic důležitého, co by nebylo jinde, v něm není). I kdybychom chtěli adresář překopírovat či přesunout, raději to nedělejme - lepší je vytvořit nové identické virtuální prostředí (při přesunu by se mohla nabořit vazba s mateřskou instalací).  
Virtuální prostředí mají ze svého principu jednu nevýhodu - pokud nějaký balíček potřebujeme v několika z nich, musíme ho do každého prostředí nainstalovat. To ve výsledku znamená, že se nám po nějaké době nemusí dostávat místo na disku. Proto může být účelné prostředí v nepoužívaných projektech vymazat s tím, že v případě nutnosti ona prostředí znovuvytvoříme s pomocí requirements souborů.

## Requirements
Soubor requirements.txt v minimální variantě obsahuje jména balíčků (co řádek, to jeden balíček), které by měly být pro potřeby určitého projektu nainstalovány. Namísto toho, aby člověk instaloval balíčky ručně jeden po druhém, stačí pustit
```
pip install -r requirements.txt
```
Pozn.: soubor se seznamem balíčků se může jmenovat i jinak, jméno "requirements.txt" je zkrátka jen nejobvyklejší.  
Obvykle chceme, aby virtuální prostředí vytvořené s pomocí requirements.txt bylo totožné jako to, na kterém jsme na našich počítačích původně pracovali.  V takovém případě nám ale nestačí pouze jména balíčků, měli bychom udat i jejich verze. Tj. řádky v requirements.txt by měly vypadat nějak takto:
```
pandas==2.2.2
```
V případě, že by nám z nějakého důvodu stačilo vynucovat minimální verzi balíčku, napíšeme "pandas>=2.0.0", pokud naopak nové verze nechceme, použijeme zápis "pandas<2.0.0".  
Vyvstává tu ale jeden problém. Pakliže bychom do requirements.txt například napsali jen pandy, budou se prerekvizitní balíčky (numpy, six atd.) instalovat podle pandích requirements.txt (či spíše pandího setup.py). Dost možná se tyto balíčky nainstalují v novější verzi, tj. původní a nové virtuální prostředí nebudou stejné a tak hrozí výskyt bugů. Ok, takže si obsah našeho prostředí zafixujeme skrze pip freeze? Tj. do konzole napíšeme
```
pip freeze > requirements.txt
```
Nyní by se ale do našeho nového, dejme tomu produkčního environmentu nainstalovaly i věci typu pytest anebo třeba jupyter, které v produkci potřeba nebudou.  
Na rovinu nyní netuším, zda je lepší možnost 1, možnost 2 anebo zda neexistuje třetí cesta skrze dodatečnou utilitu typu Poetry. Možná je řešením mít krom vývojového prostředí i testovací prostředí, kde budou jen ty opravdu potřebné balíčky pro běh aplikace a do produkčního requirements pak půjde freeze právě tohoto prostředí? 

## PEP 8
Jak velké používat odřádkování (a mají se používat tabulátory či mezerníky)? Jaká by měla být maximální délka řádku? Podle jaké konvence by měly být pojmenovány proměnné? Tyto a mnohé další otázky mohou mít principielně různé odpovědi. V rámci jednoho týmu by jejich řešení mohlo vést k dlouhému dohadování, v globálním pohledu by zase různé předpoklady o tom, jak má co vypadat, mohly vést k ztížené čitelnosti kódu od různých lidí. Aby se zamezilo divergenci v konvencích, vznikl oficiální dokument věnovaný vhodné formální podobě kódu [PEP 8](https://peps.python.org/pep-0008/). Člověk též občas narazí na zmínky o [Google konvencích](https://google.github.io/styleguide/pyguide.html). Ty se možná kdysi od PEP 8 lišily, dnes se ale víceméně jedná o podrobnější rozepsání oblastí v PEP 8 zmíněných.  

No jo, ono je sice užitečné si výše uvedené dokumenty přečíst, ale kdo má při tvorbě a kontrole kódu na vše myslet? Naštěstí existují utility, které nám s takovou prací pomohou. Jedná se o formattery a lintery. Formattery hledají věci, které lze bez znalosti kontextu kódu opravit, a opravu provedou. Mluvíme tu o špatném odřádkování, chybějících či přebývajících mezerách apod. Nejznámějšími formattery jsou **Black** a **Yapf**. Lintery sice chybějící či přebývající mezery taky hledají, nicméně zaměřují se i na nepoužité proměnné či funkce, chybějící importy či nevhodná pojmenování. Změny ale neprovádějí, jen uživateli problémy nahlásí. V oblasti pythoních linterů dominují **Pylint** a **Flake8**.

### Black
[Black](https://black.readthedocs.io/en/stable/) je význačný možnostmi nastavení - téměř žádné nejsou. 
Použití je triviální - v konzoli člověk napíše
```
black muj_skript.py
```
případně
```
black adresar_se_skripty
```
Jednu z mála věcí, které se v Blacku dají změnit, představuje šířka řádku. Pokud nám defaultní velikost 88 nevyhovuje, použijeme parametr -l (resp. --line-length):
```
black adresar_se_skripty -l 100
```

### Yapf
Yapf si můžeme představit jako Black, u kterého se dá změnit úplně všechno.  
Pro základní úpravu jednoho souboru musíme použít parametr -i (resp. --in-place) - bez něj by se výstup Yapfu neuložil do souboru, nýbrž by se pouze vytiskl do konzole.
```
yapf -i muj_skript.py
```
Pokud chceme upravit všechny soubory v adresáři, musíme navíc přidat parametr -r (--recursive)
```
yapf -i -r adresar_se_skripty
```
Defaultně Yapf postupuje podle PEP 8. Chceme-li, aby se pracovalo podle Google stylu, použijeme parametr --style následovaný hodnotou "google":
```
yapf -i --style google muj_skript.py
```
Pokud bychom z nějakého důvodu chtěli defaultní chování specifikovat explicitně, použijeme namísto toho hodnotu "pep8". Můžeme si vytvořit i svůj vlastní styl. Ten popíšeme v konfiguračním souboru (v příkladu pojmenovaném jako .yapfstyle, ale jmenovat se může všelijak). Příklad budiž takovýto:
```
[style]
based_on_style = pep8
column_limit = 40
```
Zde specifikujeme, z jakého stylu vycházíme a následně uvádíme změny - v příkladu maximální šířku řádku rovnou 40 znakům. Jména parametrů jsou k nalezení v [dokumentaci](https://github.com/google/yapf). Při samotném použití napíšeme jméno konfiguráku za parametr --style:
```
yapf -i --style .yapfstyle muj_skript.py
```

### Pylint
Pylintem zkontrolujeme pythoní skript pomocí příkazu
```
pylint muj_skript.py
```
Pylint můžeme vypustit i na celý adresář:
```
pylint adresar_se_skripty
```
Vždy dostaneme report věující se postupně různým okruhům problémů, který je ukončen bodovým hodnocením. Bodové maximum reprezentující z hlediska Pylintu ideální kód se rovná 10, minimum může být v principu v mínus nekonečnu (samozřejmě by se člověk musel hodně snažit, aby skončil pod dejme tomu -10). Též se uživateli ukáže bodové hodnocení předchozího běhu, aby mohl posoudit, jestli se kvalita kódu s provedenými změnami spíše zlepšila či zhoršila.  

Představme si nyní, že bychom Pylint nepoušteli ručně v rámci nějaké ad-hoc kontroly, nýbrž že by byl součástí CICD pipeliny. Jak docílit toho, aby se v případě nízkého skóre nasazovací pipelina zastavila? I když to při ruční práci nevidíme, Pylint krom reportu jako hádám každý proces vrací *exit code*. Nicméně v tomto případě číslo různé od nuly neznamená, že Pylint jako program spadl, nýbrž že ve zkoumaném skriptu nalezl nějaké problémy. A jaké? To je definované kódem určeným s pomocí [této](https://pylint.pycqa.org/en/latest/user_guide/usage/run.html#exit-codes) tabulky. Jestli správně chápu její logiku, tak výskyt aspoň jednoho problému určitého druhu přispívá odpovídajícím číslem z tabulky a tato čísla se ve finále sečtou. Přitom výskyt různých problémů stejného typu už žádný dodatečný příspěvek nepřidává. Tj. například  když bychom v reportu měli tři problémy s konvencí a dva errory, vrátil by se nám *exit code* o velikosti 18. A jak vlastně onen *exit code* zobrazíme? Musíme jako příkaz následující po proběhnutí pylintu ve windosowské konzoli pustit následující:
```
echo %ERRORLEVEL%
```
No jo, jenže občas kód nesplňuje všechny záležitosti na 100%. A asi nedává smysl, aby se nasazení zastavilo kvůli jedné hlášce na konvenci. Proto lze Pylint pustit s parametrem --fail-under=cislo, který zajistí, že se nenulový error code vygeneruje jen když je skóre reportu menší než cislo (defaultně 10). Tj. v praxi to vypadá třeba takto:
```
pylint --fail-under=7 muj_skript.py
```



nastavení a že si ho najde sám (když má soubor správné jméno)




## Pydantic
## Lintery (Pylint, Flake8)

## Docstringy

## Typové anotace a mypy

## Bandit